#08 파이썬 추천 시스템 패키지-Surprise

In [1]:
!pip install scikit-surprise

     |████████████████████████████████| 11.8 MB 3.9 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1633718 sha256=e43bd347a7c47c0115c158d2a8ef45b60eb4b8a4c5a0ccd0651d4b18f8149f31
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


##Surprise 를 이용한 추천 시스템 구축

In [2]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

In [9]:
data = Dataset.load_builtin('ml-100k',prompt=False)
#수행 시마다 동일하게 데이터를 분할하기 위해 random_state 값 부여
trainset, testset = train_test_split(data,test_size=0.25,random_state=0)

In [10]:
algo = SVD()
algo.fit(trainset)

In [11]:
predictions = algo.test(testset)
print('prediction type : ', type(predictions),'size:',len(predictions))
print('prediction 결과의 최초 5개 추출')
predictions[:5]

prediction type :  <class 'list'> size: 25000
prediction 결과의 최초 5개 추출


[Prediction(uid='120', iid='282', r_ui=4.0, est=3.6669964294730297, details={'was_impossible': False}),
 Prediction(uid='882', iid='291', r_ui=4.0, est=3.8126074619434096, details={'was_impossible': False}),
 Prediction(uid='535', iid='507', r_ui=5.0, est=4.0463807147498905, details={'was_impossible': False}),
 Prediction(uid='697', iid='244', r_ui=5.0, est=3.362789881463163, details={'was_impossible': False}),
 Prediction(uid='751', iid='385', r_ui=4.0, est=3.246904985783467, details={'was_impossible': False})]

In [13]:
[(pred.uid,pred.iid,pred.est) for pred in predictions[:3]]

[('120', '282', 3.6669964294730297),
 ('882', '291', 3.8126074619434096),
 ('535', '507', 4.0463807147498905)]

In [14]:
#사용자 아이디, 아이템 아이디는 문자열로 입력해야 함.
uid = str(196)
iid = str(302)
pred = algo.predict(uid,iid)
print(pred)

user: 196        item: 302        r_ui = None   est = 4.07   {'was_impossible': False}


In [15]:
accuracy.rmse(predictions)

RMSE: 0.9490


0.9490297294819476

In [4]:
import pandas as pd

ratings = pd.read_csv('/content/drive/MyDrive/ESAA(22-1)/Week11/movielens/ratings.csv')
#ratings_noh.csv 파일로 업로드 시 인덱스와 헤더를 모두 제거한 새로운 파일 생성
ratings.to_csv('/content/drive/MyDrive/ESAA(22-1)/Week11/movielens/ratings_noh.csv',index=False,header=False)

In [ ]:
from surprise import Reader
reader = Reader(line_format='user item rating timestamp',sep=',',rating_scale=(0.5,5))
data = Dataset.load_from_file('/content/drive/MyDrive/ESAA(22-1)/Week11/movielens/ratings_noh.csv',reader=reader)

In [ ]:
trainset,testset = train_test_split(data,test_size=0.25,random_state=0)

#수행 시마다 동일한 결과를 도출하기 위해 random_state 설정
algo = SVD(n_factors=50,random_state=0)

#학습 데이터 세트로 학습하고 나서 테스트 데이터 세트로 평점 예측 후 RMSE 평가
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8908


0.8907754769926038

##판다스 DataFrame 에서 Surprise 데이터 세트로 로딩

In [ ]:
import pandas as pd
from surprise import Reader, Dataset

In [ ]:
ratings = pd.read_csv('/content/drive/MyDrive/ESAA(22-1)/Week11/movielens/ratings.csv')
reader = Reader(rating_scale=(0.5,5.0))

#ratings DataFrame 에서 칼럼은 사용자 아이디, 아이템 아이디, 평점 순서를 지켜야 합니다.
data = Dataset.load_from_df(ratings[['userId','movieId','rating']],reader)
trainset,testset = train_test_split(data,test_size=0.25,random_state=0)

algo = SVD(n_factors=50,random_state=0)
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8908


0.8907754769926038

##Surprise 추천 알고리즘 클래스

<Surprise 에서 추천 예측을 위해 자주 사용되는 추천 알고리즘 클래스>  


*   SVD : 행렬 분해를 통한 잠재 요인 협업 필터링을 위한 SVD 알고리즘
*   KNNBasic : 최근접 이웃 협업 필터링을 위한 KNN 알고리즘
*   BaselineOnly : 사용자 Bias 와 아이템 Bias 를 감안한 SGD 베이스라인 알고리즘



##베이스 라인 평점

베이스라인 평점(Baseline Rating) : 냉정한 평가를 위해 아이템 평가에 편향성(bias) 요소를 반영하여 평점을 부과하는 것  
베이스라인 평점 = 전체 평균 평점 + 사용자 편향 점수 + 아이템 편향 점수



*   전체 평균 평점 = 모든 사용자의 아이템에 대한 평점을 평균한 값
*   사용자 편향 점수 = 사용자별 아이템 평점 평균 값 - 전체 평균 평점
*   아이템 편향 점수 = 아이템별 평점 평균 값 - 전체 평균 평점





예를 들어, 모든 사용자의 평균적인 영화 평점이 3.5(전체 평균 평점:3.5), '어벤져스 3편'을 모든 사용자가 평균적으로 평점 4.2로 평가했다면 영화 평가를 늘 깐깐하게 하는 사용자 A가 '어벤져스 3편'을 어떻게 평가할 것인지 예상해보자  
모든 사용자의 평균 영화 평점(3.5) + 사용자 편향 점수(3.0-3.5=-0.5) + 아이템 편향 점수(4.2-3.5=0.7) = 3.7

##교차 검증과 하이퍼 파라미터 튜닝

In [ ]:
from surprise.model_selection import cross_validate

#판다스 DataFrame에서 Surprise 데이터 세트로 데이터 로딩
ratings = pd.read_csv('/content/drive/MyDrive/ESAA(22-1)/Week11/movielens/ratings.csv')  #reading data in pandas df
reader = Reader(rating_scale=(0.5,5.0))
data = Dataset.load_from_df(ratings[['userId','movieId','rating']],reader)

algo = SVD(random_state=0)
cross_validate(algo,data,measures=['RMSE','MAE'],cv=5,verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8942  0.8863  0.9033  0.8955  0.9024  0.8963  0.0062  
MAE (testset)     0.6905  0.6848  0.6934  0.6901  0.6933  0.6904  0.0032  
Fit time          5.87    6.19    5.19    5.15    5.41    5.56    0.40    
Test time         0.29    0.14    0.30    0.14    0.15    0.21    0.07    


{'fit_time': (5.873060941696167,
  6.185137987136841,
  5.190890073776245,
  5.153280019760132,
  5.410163879394531),
 'test_mae': array([0.69051822, 0.68475068, 0.69342953, 0.69006506, 0.69330339]),
 'test_rmse': array([0.89423311, 0.88626534, 0.90331436, 0.89553688, 0.90237178]),
 'test_time': (0.2891230583190918,
  0.14344263076782227,
  0.30486488342285156,
  0.1432805061340332,
  0.14976835250854492)}

cross_validatae() : 폴드별 성능 평가 수치와 전체 폴드의 평균 성능 수치를 함께 보여준다.)

In [ ]:
from surprise.model_selection import GridSearchCV

#최적화할 파라미터를 딕셔너리 형태로 지정
param_grid = {'n_epochs':[20,40,60],'n_factors':[50,100,200]}

#CV를 3개 폴드 세트로 지정, 성능 평가는 rmse,mse로 수행하도록 GridSearchCV 구성
gs = GridSearchCV(SVD, param_grid, measures=['rmse','mae'],cv=3)
gs.fit(data)

#최고 RMSE Evaluation 점수와 그때의 하이퍼 파라미터
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

0.8994404277969686
{'n_epochs': 20, 'n_factors': 50}
